### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
df_school_data = pd.read_csv(school_data_to_load)
df_student_data = pd.read_csv(student_data_to_load)

# Add student passing/fail to student data dataframe
student_passing_reading = []

for score in df_student_data['reading_score']:
    if score > 69:
        student_passing_reading.append(1)
    else:
        student_passing_reading.append(0)
        
df_student_data['student_passing_reading'] = student_passing_reading 

student_passing_math = []

for score in df_student_data['math_score']:
    if score > 69:
        student_passing_math.append(1)
    else:
        student_passing_math.append(0)
        
df_student_data['student_passing_math'] = student_passing_math 

student_overall_passing_nbr = df_student_data["student_passing_math"] + df_student_data["student_passing_reading"]
df_student_data["student_overall_passing"] = student_overall_passing_nbr
df_student_data['student_overall_passing'] = np.where(
   (df_student_data['student_overall_passing'] <=1), 0, 1)

# Combine the data into a single dataset.  
df_school_data_complete = pd.merge(df_student_data, df_school_data, how="left", on=["school_name", "school_name"])
df_school_data_complete


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,student_passing_reading,student_passing_math,student_overall_passing,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,1,0,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,1,0,0,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,1,0,0,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,0,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1,1,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,1,1,1,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,1,1,1,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,1,1,1,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,1,1,1,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#calculate variables
district_school_count = len(df_school_data_complete["school_name"].unique())
district_student_count = len(df_school_data_complete["Student ID"].unique())
district_total_budget = df_school_data["budget"].sum()
district_average_math_score = df_school_data_complete["math_score"].mean()
district_average_reading_score = df_school_data_complete["reading_score"].mean()

#get passing math score df
df_passing_math = df_school_data_complete.loc[df_school_data_complete["math_score"] > 69]
district_student_count_passing_math = len(df_passing_math["Student ID"].unique())
district_student_percent_passing_math = (district_student_count_passing_math / district_student_count) * 100

#get passing reading score df
df_passing_reading = df_school_data_complete.loc[df_school_data_complete["reading_score"] > 69]
district_student_count_passing_reading = len(df_passing_reading["Student ID"].unique())
district_student_percent_passing_reading = (district_student_count_passing_reading / district_student_count) * 100

#merge passing math/reading for overall values
df_district_overall = pd.merge(df_passing_math, df_passing_reading, on="Student ID")
district_overall_count = len(df_district_overall["Student ID"].unique())
district_overall_percent = (district_overall_count/district_student_count) * 100

#Formatting
district_student_count = "{:,}".format(district_student_count)
district_total_budget = '${:,.2f}'.format(district_total_budget)
district_average_math_score = '{:,.2f}'.format(district_average_math_score)
district_average_reading_score = '{:,.2f}'.format(district_average_reading_score)
district_student_percent_passing_math = '{:,.2f}%'.format(district_student_percent_passing_math)
district_student_percent_passing_reading = '{:,.2f}%'.format(district_student_percent_passing_reading)
district_overall_percent = '{:,.2f}%'.format(district_overall_percent)

df_district_summary = pd.DataFrame({"Total Schools":[district_school_count],
                                 "Total Students":[district_student_count],
                                 "Total Budget":[district_total_budget],
                                 "Average Math Score":[district_average_math_score],
                                 "Average Reading Score":[district_average_reading_score],
                                 "% Passing Math":[district_student_percent_passing_math],
                                 "% Passing Reading":[district_student_percent_passing_reading],
                                 "% Overall Passing":[district_overall_percent]})

df_district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
df_grouped_school = df_school_data_complete.groupby(["school_name"])

school_type = df_grouped_school["type"].first()
school_student_count = df_grouped_school["Student ID"].count()
school_average_math_score = df_grouped_school["math_score"].mean()
school_average_reading_score = df_grouped_school["reading_score"].mean()
school_budget = df_grouped_school["budget"].sum()
school_percent_passing_math = df_grouped_school["student_passing_math"].sum()
school_percent_passing_math = (school_percent_passing_math/school_student_count)*100
school_percent_passing_reading = df_grouped_school["student_passing_reading"].sum()
school_percent_passing_reading = (school_percent_passing_reading/school_student_count)*100
school_overall_percent_passing = df_grouped_school["student_overall_passing"].sum()
school_overall_percent_passing = (school_overall_percent_passing/school_student_count)*100

school_budget = school_budget/school_student_count
school_per_student = school_budget/school_student_count



df_school_summary = pd.DataFrame({"School Type": school_type,
                                  "Total Students": school_student_count,
                                  "Total School Budget": school_budget,
                                  "Per Student Budget": school_per_student,
                                  "Average Math Score": school_average_math_score,
                                  "Average Reading Score": school_average_reading_score,
                                  "% Passing Math": school_percent_passing_math, 
                                  "% Passing Reading":school_percent_passing_reading,
                                  "% Overall Passing":school_overall_percent_passing})

format_mapping = {'Total School Budget': '${:,.2f}', 'Per Student Budget': '${:,.2f}','% Overall Passing': '{:.2f}%'}

for key, value in format_mapping.items():
    df_school_summary[key] = df_school_summary[key].apply(value.format)


df_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [40]:

sort_by_top_performing = df_school_summary.sort_values('school_overall_percent_passing')

KeyError: 'school_overall_percent_passing'

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type